In [23]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

from google.colab import files

print(tf.version.VERSION)


2.7.0


In [24]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [25]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [26]:
BEST_RESULT_WEIGHTS = 'my_model_weights.h5'

model.load_weights(BEST_RESULT_WEIGHTS) # Load the best weight we had from the previous prunings
def sparsify(model,t):
    tr_weights = np.copy(model.get_weights())
    stacked_wts = np.hstack([i.flatten() for i in tr_weights])
    thres = np.percentile(np.abs(stacked_wts),t)
    for i in range(len(tr_weights)):
        tr = np.abs(tr_weights[i])>thres
        tr_weights[i] = np.multiply(tr_weights[i],tr)
    return tr_weights

t=97
tr_weights = sparsify(model,t)
model.set_weights(tr_weights)


# Remasking method
initWs = [tf.identity(W) for W in model.trainable_variables]

# The following code for class definition is modelled after: 
# https://keras.io/guides/customizing_what_happens_in_fit/
class MaskModel(models.Sequential):
  '''
  A Keras model extended from the Sequential model, such that the per-epoch
  training is overwritten so that the gradient descent would only update the weights 
  initially set to nonzeros
  '''
  def __init__(self, initWs):
    '''
    initWs: List[Tensor], the initial best weights Saurabh acheievd
    '''
    super().__init__()
    self.mask = [tf.cast(initW!=0, tf.float32) for initW in initWs]
  
  def train_step(self, data):
    '''
    the per-epoch traning we overwrite
    '''
    x, y = data

    with tf.GradientTape() as tape: 
      y_pred = self(x, training=True)
      loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)   

    # Compute gradients and mask out the weights that should remain zero
    gradients = [ g*m for g, m in zip(tape.gradient(loss, self.trainable_variables), self.mask)]
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    # Update and return metrics 
    self.compiled_metrics.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}

model = MaskModel(initWs)
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
# model.load_weights(BEST_RESULT_WEIGHTS) # Again. load the best weight we had from the previous prunings
model.set_weights(tr_weights)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)


In [35]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

def getSparse(model):
  return np.sum(np.hstack([k.flatten() for k in model.get_weights()])==0)/np.shape(np.hstack([k.flatten() for k in model.get_weights()]))[0]
def getAccu(model):
  return model.evaluate(val_images, val_labels, batch_size=128)[1]
print(getSparse(model), getAccu(model))

# Train as many epoches as possible, until we have a very good validation accuracy (e.g. 75%+)
# model.fit(x = train_images, y = train_labels,batch_size=64, epochs=10,validation_data=(val_images, val_labels))
################################################################################NEW
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=40)

model.fit(x = train_images, y = train_labels,batch_size=128, epochs=200,validation_data=(val_images, val_labels),callbacks=[early_stopping, model_checkpoint_callback])##ADDED Callbacks
model.load_weights(checkpoint_filepath)##model will load the checkpointed model weights with best val_accuracy
################################################################################NEW

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


20/20 [==============================] - 2s 75ms/step - loss: 0.7029 - accuracy: 0.7469
0.969999983134688 0.746930718421936
Epoch 1/200
176/176 [==============================] - 54s 300ms/step - loss: 0.7130 - accuracy: 0.7342 - val_loss: 0.6985 - val_accuracy: 0.7442
Epoch 2/200
176/176 [==============================] - 52s 296ms/step - loss: 0.7077 - accuracy: 0.7325 - val_loss: 0.6992 - val_accuracy: 0.7438
Epoch 3/200
176/176 [==============================] - 53s 298ms/step - loss: 0.7113 - accuracy: 0.7298 - val_loss: 0.7053 - val_accuracy: 0.7406
Epoch 4/200
176/176 [==============================] - 53s 304ms/step - loss: 0.7149 - accuracy: 0.7276 - val_loss: 0.7102 - val_accuracy: 0.7358
Epoch 5/200
176/176 [==============================] - 53s 300ms/step - loss: 0.7048 - accuracy: 0.7320 - val_loss: 0.7072 - val_accuracy: 0.7414
Epoch 6/200
176/176 [==============================] - 52s 298ms/step - loss: 0.7065 - accuracy: 0.7354 - val_loss: 0.7047 - val_accuracy: 0.7426


In [36]:
a,b = getSparse(model), getAccu(model)
print(a,b,(a+b)/2)


20/20 [==============================] - 1s 74ms/step - loss: 0.6917 - accuracy: 0.7461
0.969999983134688 0.7461386322975159 0.8580693077161019


In [37]:
model.save_weights("my_model_weights.h5")
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
# TODO: Prepare data for frontier diagram